In [1]:
import os
import re
import numpy as np
from tqdm import tqdm
from bleach import clean
from bs4 import BeautifulSoup

In [2]:
root = '../Downloads/Dataset/train/'
opj = os.path.join

In [3]:
labels = list(os.listdir(root))
texts, targets = [], []

In [4]:
def prep(xml):
    soup = BeautifulSoup(xml, 'lxml')
    cn = soup.get_text().strip()
    r = re.sub(r"(?is)<(script|style).*?>.*?(</\1>)", "", cn.strip())
    # Then we remove html comments. This has to be done before removing regular
    # tags since comm\d  '[^\w]'ents can contain '>' characters.
    r = re.sub(r'[/\W+/g+\d+\s+\t]'," ",r)
    r = re.sub(r'<.*?>', " ", r)
    r = re.sub(r"(?s)<.*?>", " ", r)
    r = re.sub(r"&nbsp;", ' ', r)
    r = re.sub(r'\s+', ' ',r)
    r = re.sub(r"  ", " ", r)
    r = re.sub(r"  ", " ", r)
    r = clean(r)
    return r

In [5]:
for label in tqdm(labels, desc='Labels'):
    files = os.listdir(opj(root, label))
    paths = [opj(opj(root, label), i) for i in files]
    for path in tqdm(paths, desc='Reading Files'):
        with open(path, 'r') as fl:
            xml = fl.read()
        text = prep(xml)
        texts.append(text)
        targets.append(label)

Labels:  50%|█████     | 1/2 [00:23<00:23, 23.12s/it] 13.04it/s]


Labels: 100%|██████████| 2/2 [00:31<00:00, 15.52s/it]


In [6]:
import pandas as pd

In [7]:
text = pd.DataFrame(texts, columns = ['texts'])
targ= pd.DataFrame(targets, columns = ['targets'])
df = pd.concat([text,targ],axis = 1)
df = df.sample(frac = 1).reset_index(drop = True)

In [8]:
df.head(5)

,texts,targets
0,Cancer Ima in Cancer Ima in Cancer Ima in BioM...,cancer
1,Case Rep PsychiatryCase Rep PsychiatryCRIPSCas...,noncancer
2,Cancer Ima in CICancer Ima in Cancer Ima in e ...,cancer
3,Cancer Ima in Cancer Ima in Cancer Ima in BioM...,cancer
4,Case Reports ImmunolCase Reports ImmunolCRIICa...,noncancer


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
print('Generating vectors')
tf = TfidfVectorizer(sublinear_tf= True,norm="l2",max_features = 1500,stop_words= 'english')
x = tf.fit_transform(text)
# we know there are only two labels and so this scheme works
# otherwise use something like onehot encoder
y = np.array([(1 if i == labels[0] else 0)
              for i in targ])

Generating vectors


In [11]:
ss_tftdm = tf.transform(text)
std_n = ss_tftdm.toarray()

In [12]:
nx = pd.DataFrame(ss_tftdm.toarray(),columns = tf.get_feature_names())

In [14]:
st

<1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>